<a href="https://colab.research.google.com/github/Olfeng-xalaz/Fallstudie_SCM/blob/main/Fallstudie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# pip als Paketmanager
! pip install -q pyscipopt
! pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.2/17.2 MB 41.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
from pyscipopt import Model, quicksum

# Optimierungsmodell zur Elektrifizierung der Logistik

### Import der CSV Dateien

In [3]:
import os

# Prüfen ob in Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    # In Colab: Repository klonen (nur einmal)
    if not os.path.exists('Fallstudie_SCM'):
        !git clone https://github.com/Olfeng-xalaz/Fallstudie_SCM.git
    folder = "Fallstudie_SCM/DataCSV"
else:
    # Lokal in VS Code: Relativer Pfad
    folder = os.path.join(os.path.dirname(__file__), "DataCSV") if "__file__" in dir() else "DataCSV"

Cloning into 'Fallstudie_SCM'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 53 (delta 17), reused 10 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (53/53), 19.24 KiB | 6.41 MiB/s, done.
Resolving deltas: 100% (17/17), done.


In [4]:
chargers = pd.read_csv(f"{folder}/chargers.csv", sep=";")

In [5]:
chargers.head()

,charger_model,capex_yearly,opex_yearly,max_power,charging_spots
0,Alpitronic-50,3000,1000,50,2
1,Alpitronic-200,10000,1500,200,2
2,Alpitronic-400,16000,2000,400,2


In [6]:
dtrucks_specs = pd.read_csv(f"{folder}/diesel_trucks.csv", sep=";")

In [7]:
dtrucks_specs.head()

,truck_model,capex_yearly,opex_yearly,avg_diesel_per_100km,kfz_yearly,gross_vehicle_weight,emission_class,co2_emission_class
0,ActrosL,24000,6000,26,556,40,EURO 6,1


In [8]:
etrucks_specs = pd.read_csv(f"{folder}/electric_trucks.csv", sep=";")

In [9]:
etrucks_specs.head()

,truck_model,capex_yearly,opex_yearly,avg_energy_kWh_per_100km,thg_yearly,max_power,soc_max_kWh
0,eActros600,60000,6000,110,1000,400,621
1,eActros400,50000,5000,105,1000,400,414


In [10]:
routes = pd.read_csv(f"{folder}/routes.csv", sep=";")

In [11]:
routes.head()

,route_id,route_name,distance_total,distance_toll,starttime,endtime
0,t-4,Nahverkehr,250,150,06:45,17:15
1,t-5,Nahverkehr,250,150,06:30,17:00
2,t-6,Nahverkehr,250,150,06:00,16:30
3,s-1,Ditzingen,120,32,05:30,15:30
4,s-2,Ditzingen,120,32,06:00,16:00


### Indexmengen

In [12]:
#Erstellen einer Modellinstanz
scip = Model()

In [13]:
R = routes["route_id"].unique() # Menge der Routen
C = chargers["charger_model"].unique() # Menge der Charger-Modelle
L = pd.concat([dtrucks_specs["truck_model"],etrucks_specs["truck_model"]]).unique() # Menge der Fahrzeugmodelle
P = range(1, 21) # Potenzielle Fahrzeuge
T = range(1, 97) # 96 Zeitintervalle pro Tag
I = range(1, 4) # Potenzielle Säulen-Slots
T_night = [t for t in T if t > 18*4 or t <= 6*4] # Nachtintervalle: 18:00–06:00


print("R (Routen):", R)
print("C (Charger):", C)
print("L (Fahrzeugmodelle):", L)
print("P (potenzielle Fahrzeuge):", list(P))
print("T (Zeitintervalle):", list(T))
print("I (Säulen):", list(I))
print("T_night (Nachtintervalle):", list(T_night))


R (Routen): ['t-4' 't-5' 't-6' 's-1' 's-2' 's-3' 's-4' 'w1' 'w2' 'w3' 'w4' 'w5' 'w6'
 'w7' 'r1' 'r2' 'r3' 'h3' 'h4' 'k1']
C (Charger): ['Alpitronic-50' 'Alpitronic-200' 'Alpitronic-400']
L (Fahrzeugmodelle): ['ActrosL' 'eActros600' 'eActros400']
P (potenzielle Fahrzeuge): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
T (Zeitintervalle): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96]
I (Säulen): [1, 2, 3]
T_night (Nachtintervalle): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96]


### Parameter

In [14]:
# Chargers
chargers.set_index(["charger_model"], inplace=True)

### Entscheidungsvariablen

### Zielfunktion

### Restriktionen

### Berechnung des Ergebnisses